## Implementation of PilotNet, with variation

Implement [PilotNet](https://arxiv.org/pdf/1604.07316.pdf) using Keras (with theano backend), with various differences

In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


### Check data format

train using udacity data

check the csv output, and shuffle

In [2]:
df = pd.read_csv("data/driving_log.csv", names=['Center', 'Left', 'Right', 'Steering Angle', 'Throttle', 'Break', 'Speed'])
df = shuffle(df)
df.head(3)

,Center,Left,Right,Steering Angle,Throttle,Break,Speed
3873,data/IMG/center_2017_04_06_17_12_33_822.jpg,data/IMG/left_2017_04_06_17_12_33_822.jpg,data/IMG/right_2017_04_06_17_12_33_822.jpg,0.074324,0.268057,0.0,23.36063
7671,data/IMG/center_2017_04_06_17_43_28_190.jpg,data/IMG/left_2017_04_06_17_43_28_190.jpg,data/IMG/right_2017_04_06_17_43_28_190.jpg,0.000000,0.247335,0.0,29.99862
698,data/IMG/center_2017_04_06_17_07_57_505.jpg,data/IMG/left_2017_04_06_17_07_57_505.jpg,data/IMG/right_2017_04_06_17_07_57_505.jpg,-0.090186,0.198581,0.0,20.05411


###  First, only predict steering angle, using only center image

define input and output, and load to memory. (this may take awhile)

In [3]:
X_train = np.array([img_path_to_arr(p) for p in df['Center']])
y_train = df['Steering Angle'].values

input_shape = X_train.shape[1:]

In [8]:
reduce_lr = ReduceLROnPlateau(patience=4, verbose=1)

In [5]:
def get_model_original():
    model = Sequential([
        Lambda(lambda x: x / 255.0 - 0.5, input_shape=input_shape, output_shape=input_shape),
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu'),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(50, activation='relu'),
        Dense(10, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer="adam", loss="mse") 
    model.summary()
    return model

In [6]:
model_original = get_model_original()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 160, 320)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 78, 158)       1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 37, 77)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 17, 37)        43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 15, 35)        27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 13, 33)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 27456)             0         
__________

In [7]:
model_original.fit(X_train, y_train, validation_split = 0.2, shuffle = True, epochs = 20, callbacks = [reduce_lr])

Train on 9742 samples, validate on 2436 samples
Epoch 1/20
9742/9742 [==============================] - 10s - loss: 0.0187 - val_loss: 0.0186
Epoch 2/20
9742/9742 [==============================] - 10s - loss: 0.0155 - val_loss: 0.0176
Epoch 3/20
9742/9742 [==============================] - 10s - loss: 0.0146 - val_loss: 0.0149
Epoch 4/20
9742/9742 [==============================] - 10s - loss: 0.0138 - val_loss: 0.0148
Epoch 5/20
9742/9742 [==============================] - 10s - loss: 0.0127 - val_loss: 0.0143
Epoch 6/20
9742/9742 [==============================] - 10s - loss: 0.0117 - val_loss: 0.0148
Epoch 7/20
9742/9742 [==============================] - 10s - loss: 0.0100 - val_loss: 0.0137
Epoch 8/20
9742/9742 [==============================] - 10s - loss: 0.0087 - val_loss: 0.0151
Epoch 9/20
9742/9742 [==============================] - 10s - loss: 0.0072 - val_loss: 0.0145
Epoch 10/20
9742/9742 [==============================] - 10s - loss: 0.0054 - val_loss: 0.0136
Epoch 11/20

As we can see above, we're starting to overfit around epoch 10.

### Use batch normalization

Instead of normalizing the input ourselves, let's add a batch normalization layer as the first layer.

While we're at it, let's add bactch norm after each layer.

In [9]:
def get_model_batch_norm():
    model = Sequential([
        BatchNormalization(axis=1, input_shape=input_shape),
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(50, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='relu'),
        BatchNormalization(),
        Dense(1)
    ])
    
    model.compile(optimizer="adam", loss="mse") 
    model.summary()
    return model

In [10]:
model_batch_norm = get_model_batch_norm()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 3, 160, 320)       12        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 78, 158)       1824      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 78, 158)       96        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 36, 37, 77)        21636     
_________________________________________________________________
batch_normalization_3 (Batch (None, 36, 37, 77)        144       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 48, 17, 37)        43248     
_________________________________________________________________
batch_normalization_4 (Batch (None, 48, 17, 37)        192       
__________

In [12]:
model_batch_norm.fit(X_train, y_train, validation_split = 0.2, shuffle = True, epochs = 20, callbacks = [reduce_lr])

Train on 9742 samples, validate on 2436 samples
Epoch 1/20
9742/9742 [==============================] - 21s - loss: 0.0624 - val_loss: 0.0286
Epoch 2/20
9742/9742 [==============================] - 21s - loss: 0.0183 - val_loss: 0.0167
Epoch 3/20
9742/9742 [==============================] - 21s - loss: 0.0146 - val_loss: 0.0162
Epoch 4/20
9742/9742 [==============================] - 21s - loss: 0.0127 - val_loss: 0.0157
Epoch 5/20
9742/9742 [==============================] - 21s - loss: 0.0102 - val_loss: 0.0155
Epoch 6/20
9742/9742 [==============================] - 21s - loss: 0.0081 - val_loss: 0.0165
Epoch 7/20
9742/9742 [==============================] - 21s - loss: 0.0068 - val_loss: 0.0159
Epoch 8/20
9742/9742 [==============================] - 21s - loss: 0.0058 - val_loss: 0.0147
Epoch 9/20
9742/9742 [==============================] - 21s - loss: 0.0051 - val_loss: 0.0154
Epoch 10/20
9742/9742 [==============================] - 21s - loss: 0.0048 - val_loss: 0.0150
Epoch 11/20

In [13]:
model_batch_norm.fit(X_train, y_train, validation_split = 0.2, shuffle = True, epochs = 20, callbacks = [reduce_lr])

Train on 9742 samples, validate on 2436 samples
Epoch 1/20
9742/9742 [==============================] - 21s - loss: 0.0036 - val_loss: 0.0143
Epoch 2/20
9742/9742 [==============================] - 21s - loss: 0.0028 - val_loss: 0.0143
Epoch 3/20
9742/9742 [==============================] - 21s - loss: 0.0026 - val_loss: 0.0141
Epoch 4/20
9742/9742 [==============================] - 21s - loss: 0.0023 - val_loss: 0.0139
Epoch 5/20
9742/9742 [==============================] - 21s - loss: 0.0021 - val_loss: 0.0139
Epoch 6/20
9742/9742 [==============================] - 21s - loss: 0.0019 - val_loss: 0.0141
Epoch 7/20
9742/9742 [==============================] - 21s - loss: 0.0019 - val_loss: 0.0139
Epoch 8/20
9742/9742 [==============================] - 21s - loss: 0.0018 - val_loss: 0.0139
Epoch 9/20
9728/9742 [============================>.] - ETA: 0s - loss: 0.0017
Epoch 00008: reducing learning rate to 1.0000000475e-05.
9742/9742 [==============================] - 21s - loss: 0.0017 

As we can see, batch norm didn't help.